## Upload ENS Data to Cyber Protocol

In [1]:
import pandas as pd
from numpy import ndarray
from ens import ENS
from web3 import Web3, HTTPProvider
from tqdm.notebook import tqdm
import ipfshttpclient
import content_hash
import requests
import json
from pprint import pprint
import warnings

from config import ETH_URL, MN
from src.contract_utils import execute_contract_bash


EXTRACT_NAMES = False
EXTRACT_ENS_DATA = False
AGGREGATE_ENS_DATA = True

SUBGRAPH_CONTRACT_NAME_TO_CONTENT = 'pussy1hrpna9v7vs3stzyd4z3xf00676kf78zpe2u5ksvljswn2vnjp3ysehgnqr'
SUBGRAPH_CONTRACT_NAME_TO_OWNER = 'pussy1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3s79e9sa'
SUBGRAPH_CONTRACT_NAME_TO_ASSOCIATED = 'pussy17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs9qsjlf'

tqdm.pandas()

w3 = Web3(HTTPProvider(ETH_URL))
ns = ENS.fromWeb3(w3)
ipfs_client = ipfshttpclient.connect()

/Users/sergenedashkovsky/Library/Python/3.9/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.15.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Items with content CIDs

In [2]:
ens_data_with_cid_df = pd.read_csv('data/ens_data_with_content_cid.csv', index_col=0)
ens_data_with_cid_df.head(10)

,name,owner,resolver_address,associated_address,contenthash,content_cid,name_cid,owner_cid,associated_address_cid
66336,0-1337.eth,0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,NaN,e30101701220d6f0e0d157170f52052e3f81bdafafb2e3...,QmcogzR9ouQJDgsiaTWWze4ndMLkn6gewRqKVWeKXVozh7,QmUwjXxYcgk1j2Eh7jiVtgSGn3hmLHWoqNDLTonts6hJV8,QmQk5wC2ZVcHme63hyuc4jhPtZA2z3c9YdazqqTQGZEkfH,NaN
71908,0000x.eth,0x77C9e49073d9830723195f3788570c2c36c97688,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x77C9e49073d9830723195f3788570c2c36c97688,e301017012203c7939754cbfabd4a7932a740b2af3ced2...,QmSQiPAPaT5E9y41VgHhc5kFRhRFSvZ95d5sBzH6VJwdPo,QmVzMe3mUGLB68Lz5wpQcfjab8akjQj7SdoE1ioAzfiw6E,QmZnQD7eayt3mZ1F6KcEWXKnubAd6igHC5ekjXhzVWVDsW,QmZnQD7eayt3mZ1F6KcEWXKnubAd6igHC5ekjXhzVWVDsW
75760,0008a.eth,0x88441DCD4Bc985965e69ecf5A454B04DC84eD876,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x88441DCD4Bc985965e69ecf5A454B04DC84eD876,e30101701220cdaf3eb20495dacca5225a9a0838207af7...,QmcBZGuZCQsd5zuumVRuLhXnhkWjWJiXtmcE9XQVsAXV8N,Qmawaesea9bHkCMmZupyMn5pbVgAJjKVM3MS2rUqZ1GhKf,QmRqj1dW29fnzY3fakYNr4tiqsiGF7LDSRsMb5cNbBv2BU,QmRqj1dW29fnzY3fakYNr4tiqsiGF7LDSRsMb5cNbBv2BU
78545,001x0.eth,0x338571a641d8c43F9e5A306300c5d89E0CB2CFAF,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x338571a641d8c43F9e5A306300c5d89E0CB2CFAF,e301017012209668df055d7566e1631a6db9d66015204a...,QmYTncViYfeLbyHS6EECfUoRW3kjqrbiUmKbZLZEXUmA6R,QmXk8AcZzo88TBa1TiJT3uLzcKnTY8DTyVa7euFmMh9qYw,QmYKYMxyQ1HopockUxdkxpLmq1qM4th9QvKSh6BmsVXF1q,QmYKYMxyQ1HopockUxdkxpLmq1qM4th9QvKSh6BmsVXF1q
79540,002511.eth,0x4E99f6Fdb8B5a3A292aa2194c970c2A612469Fe2,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x4E99f6Fdb8B5a3A292aa2194c970c2A612469Fe2,e301017012205952f152144216dfc3267c1f44130f1235...,QmUMLME8Lkipg1cqw8m3pwpWHhRN34ioFEbjbhw46oqkN2,QmNbqwb5T2oWTUjWMRWxUBAGYUvy3v6Ta7SHaz83Yo9TT6,QmYguMgvyupm56PwfM4D1ci16sg16rkkkfBcto2tWd2t2J,QmYguMgvyupm56PwfM4D1ci16sg16rkkkfBcto2tWd2t2J
79628,00258.eth,0x9e6D2A36fa88E5123fD09d22F00e4E9ca8669999,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x9e6D2A36fa88E5123fD09d22F00e4E9ca8669999,e301017012200429cb961fb23fd71947c477c881f63671...,QmNcuJbp2YLK6BmhdTniVsjjfxcgP16dGBKZeUHQQ6vHqB,QmUhnpZg3Wvz3eSTrAj66xVa3xrKb62CF63V5zWwu6Z3pa,QmVGYpBjDxnbYJiLogFAQeo9AQcZrAx8EvwcK3pV7EZkof,QmVGYpBjDxnbYJiLogFAQeo9AQcZrAx8EvwcK3pV7EZkof
89745,0089.eth,0xFebfecE019b5E2e42C3839dCbEA87EEDac9ac365,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0xFebfecE019b5E2e42C3839dCbEA87EEDac9ac365,e301017012200676503f43d887c1e1d20c23d988d4d941...,QmNmsnw5FpncDeUvJ3ZRFY3YtvEYWoCHWvgjEwjV41gUjM,QmSTGTAnKY5n3Xz4DEY1oWEmrVPuAXTp469L4bGVWyiX12,QmQukSX9AVEr7F4Jp4zKivhzxmpoZEEnEhiZqiWkEyhKKF,QmQukSX9AVEr7F4Jp4zKivhzxmpoZEEnEhiZqiWkEyhKKF
96279,0123a.eth,0x9aE6B470D238a4ab42cAc5fEB27CAA5A19F89510,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x9aE6B470D238a4ab42cAc5fEB27CAA5A19F89510,e30101701220c801aa770d8313b86fe43cb08472166a71...,QmboPirQwrzimcSfGbeRHxySy4sH19ZmoJywVSf7bH7n8T,QmaUedgdJdVrzAEGud28YFNAv6cuwnbdxBgG4WktfZNpSZ,QmV4WBsX3F7GRyFq2R6vMU9mXGDzxDxZwNgPt1oTBxZqgH,QmV4WBsX3F7GRyFq2R6vMU9mXGDzxDxZwNgPt1oTBxZqgH
97220,014°.eth,0x70635102CF3B7D32FE0a1221b412Cb4b172A2b2f,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x70635102CF3B7D32FE0a1221b412Cb4b172A2b2f,e3010170122086f1db4d21efaa7220ad3ec4460ac2b97b...,QmXRREtZRWsppmHwJqFYW9RfazaWh5mob5iRBgX1GLtcF9,QmWRhM2YPtfZbLNaW4Fk2JLuQBBFWNXDzWyzrYazYieE4r,QmTZkZP7iuth5yrevRAwLwcQL4JjSk78rypc9mdTETRTRA,QmTZkZP7iuth5yrevRAwLwcQL4JjSk78rypc9mdTETRTRA
99057,01x01.eth,0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02,e30101701220d6f0e0d157170f52052e3f81bdafafb2e3...,QmcogzR9ouQJDgsiaTWWze4ndMLkn6gewRqKVWeKXVozh7,QmSF6D6bPeR4FF8MiJgYYY5prufPh3B17g2bmpxZkFzWqy,QmQk5wC2ZVcHme63hyuc4jhPtZA2z3c9YdazqqTQGZEkfH,QmQk5wC2ZVcHme63hyuc4jhPtZA2z3c9YdazqqTQGZEkfH


#### ENS Namespace Contracts

- name -> owner
- owner -> name
- name -> associated_address
- associated_address -> name
- name -> cid
- cid -> name

Upload ENS items with CIDs only

In [3]:
name_owner_cnt = len(ens_data_with_cid_df[~ens_data_with_cid_df.owner.isna()])
name_associated_address_cnt = len(ens_data_with_cid_df[~ens_data_with_cid_df.associated_address.isna()])
name_contenthash_cnt = len(
    ens_data_with_cid_df)
name_cid_cnt = len(
    ens_data_with_cid_df)
print(f"name -> owner:              {name_owner_cnt:>7,} ")
print(f"owner -> name:              {name_owner_cnt:>7,} \n")
print(f"name -> associated address: {name_associated_address_cnt:>7,} ")
print(f"associated address -> name: {name_associated_address_cnt:>7,} \n")
print(f"name -> content cid:        {name_cid_cnt:>7,} ")
print(f"content cid -> name:        {name_cid_cnt:>7,} \n")
print(f"Total cyberlinks:           {(name_owner_cnt + name_associated_address_cnt + name_cid_cnt) * 2 :>7,}")

name -> owner:                5,651 
owner -> name:                5,651 

name -> associated address:   5,301 
associated address -> name:   5,301 

name -> content cid:          5,651 
content cid -> name:          5,651 

Total cyberlinks:            33,206


In [4]:
print(f"owner != associated address: {len(ens_data_with_cid_df[(~ens_data_with_cid_df.associated_address.isna()) & (ens_data_with_cid_df.associated_address != ens_data_with_cid_df.owner)]):>,}")

owner != associated address: 757


### Upload to Subgraphs
Subgraph contracts
- [source code](https://github.com/cybercongress/cw-cybergift/tree/main/contracts/cw-cyber-subgraph)
- [code #3](https://cyb.ai/codes/3) in Bostrom
- [code #3](https://space-pussy.cyb.ai/codes/3) in Space Pussy

Example of bash initiation:
```bash
export INIT='{"admins":["'"$WALLET"'"], "executers":["'"$LOADWALLET"'"]}'
pussy tx wasm instantiate 3 $INIT --label 'test ens content subgraph' --admin $WALLET --from $WALLET --chain-id space-pussy --gas 3000000 --broadcast-mode block  $SNODE
```

Example of bash execution:
```bash
export EXECUTE = '{"cyberlink": {"links":[{"from":"Qmex9kMduo4U8k8AB4otfqgHRehUNs8YCksUjBQXW9WeQi","to":"QmSkCKtKSmcsa2r5ju5SnHiLysktDqFWzTHdG4QS6yTeCH"}]}}'
pussy tx wasm execute pussy1hrpna9v7vs3stzyd4z3xf00676kf78zpe2u5ksvljswn2vnjp3ysehgnqr $EXECUTE --from $LOADWALLET --chain-id space-pussy --gas 3000000 --broadcast-mode block  $SNODE
```

In [5]:
def create_cyberlinks(
        cyberlinks: ndarray,
        subgraph_contract_address: str,
        batch_size: int = 1,
        from_address: str = 'testload',
        gas: int = 10_000_000) -> tuple[list, bool]:

    def _batch(x, batch_size: int) -> list:
        return [x[_i:_i + batch_size] for _i in range(0, len(x), batch_size)]

    _res = []
    _res_status = True
    for _ens_data_with_cid_item in tqdm(_batch(cyberlinks, batch_size)):
        if len([_ for _item in _ens_data_with_cid_item if _item[0] == _item[1]]):
            warnings.warn(
                'Self-linking {}. These links have been ignored'.format(
                    ', '.join(
                            f'{{"from": "{_item[0]}", "to": "{_item[1]}"}}'
                            for _item in _ens_data_with_cid_item if _item[0] == _item[1]))
                )
        _links_item = ', '.join(
            f'{{"from": "{_item[0]}", "to": "{_item[1]}"}}' for _item in _ens_data_with_cid_item
            if _item[0] != _item[1])
        if _links_item == '':
            return _res, _res_status
        _res_item = execute_contract_bash(
            execute_query=f'''{{"cyberlink": {{"links": [{_links_item}]}}}}''',
            contract_address=subgraph_contract_address,
            from_address=from_address,
            display_data=False,
            gas=gas
        )
        _res.append(_res_item)
        if json.loads(json.loads(_res_item)['raw_log'])[0]['events'][0]['type'] == 'execute':
            print(f'Error in {len(_res) - 1}\n{{"cyberlinks": [{_links_item}]}}\n')
            _res_status = False
            break
    return _res, _res_status


def get_created_links(contract: str) -> dict:
    _txs = requests.get(f'https://lcd.space-pussy.cybernode.ai/txs?message.module=wasm&limit&execute._contract_address={contract}&limit=1000000000').json()
    _page_total = int(_txs['page_total'])

    _created_links_row_list = []
    for _page in tqdm(range(1, _page_total + 1)):
        _txs = requests.get(f'https://lcd.space-pussy.cybernode.ai/txs?message.module=wasm&limit&execute._contract_address={contract}&limit=1000000000&page={_page}').json()
        _created_links_row_list.extend(
            [{_link['key']: _link['value']}
             for _tx in _txs['txs']
             for _event in _tx['logs'][0]['events'] if _event['type'] == 'cyberlink'
             for _link in _event['attributes'] if _link['key'] != 'neuron'])

    return {_created_links_row_list[i * 2]['particleFrom']: _created_links_row_list[i * 2 + 1]['particleTo']
            for i in range(len(_created_links_row_list)//2)}

#### Upload to `name -> content CID` Subgraph contract
[test ens content subgraph](https://space-pussy.cyb.ai/contracts/pussy1hrpna9v7vs3stzyd4z3xf00676kf78zpe2u5ksvljswn2vnjp3ysehgnqr)

In [6]:
created_links_name_to_content_dict = get_created_links(
    contract=SUBGRAPH_CONTRACT_NAME_TO_CONTENT)
print(f'Created links dict length: {len(created_links_name_to_content_dict.keys()):>,}')

ens_data_with_cid_df.loc[:, 'created_link_status_name_to_content'] = \
    ens_data_with_cid_df.apply(
        lambda x: x.name_cid in created_links_name_to_content_dict.keys(),
        axis=1)

  0%|          | 0/7 [00:00<?, ?it/s]

Created links dict length: 6,560


In [7]:
res, res_status = create_cyberlinks(
    subgraph_contract_address=SUBGRAPH_CONTRACT_NAME_TO_CONTENT,
    cyberlinks=ens_data_with_cid_df[ens_data_with_cid_df.created_link_status_name_to_content == False][['name_cid', 'content_cid']].values,
    batch_size=50)
if not res_status:
    pprint(json.loads(json.loads(res[-1])['raw_log'])[0])

  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/_r/0gtnrvss25qf6227jb63hqf00000gn/T/ipykernel_40743/2128561999.py:15: UserWarning: Self-linking {"from": "QmZF8fjKwgn9P9dRHKGLzoaD2vGzreeQ2J98XTf78qAajh", "to": "QmZF8fjKwgn9P9dRHKGLzoaD2vGzreeQ2J98XTf78qAajh"}. These links have been ignored
  warnings.warn(


#### Upload to `name -> owner address` Subgraph contract
[test ens owner subgraph](https://space-pussy.cyb.ai/contracts/pussy1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3s79e9sa)

In [ ]:
created_links_name_to_owner_dict = get_created_links(
    contract=SUBGRAPH_CONTRACT_NAME_TO_OWNER)
print(f'Created links dict length: {len(created_links_name_to_content_dict.keys()):>,}')

ens_data_with_cid_df.loc[:, 'created_link_status_name_to_owner'] = \
    ens_data_with_cid_df.apply(
        lambda x: x.name_cid in created_links_name_to_owner_dict.keys(),
        axis=1)

In [ ]:
res, res_status = create_cyberlinks(
    subgraph_contract_address=SUBGRAPH_CONTRACT_NAME_TO_CONTENT,
    cyberlinks=ens_data_with_cid_df[ens_data_with_cid_df.created_link_status_name_to_owner == False][['name_cid', 'owner_cid']].values,
    batch_size=50)
if not res_status:
    pprint(json.loads(json.loads(res[-1])['raw_log'])[0])

#### Upload to `name -> associated address` Subgraph contract

In [ ]:
created_links_name_to_associated_dict = get_created_links(
    contract=SUBGRAPH_CONTRACT_NAME_TO_ASSOCIATED)
print(f'Created links dict length: {len(created_links_name_to_content_dict.keys()):>,}')

ens_data_with_cid_df.loc[:, 'created_link_status_name_to_associated'] = \
    ens_data_with_cid_df.apply(
        lambda x: x.name_cid in created_links_name_to_associated_dict.keys(),
        axis=1)

In [ ]:
res, res_status = create_cyberlinks(
    subgraph_contract_address=SUBGRAPH_CONTRACT_NAME_TO_ASSOCIATED,
    cyberlinks=ens_data_with_cid_df[ens_data_with_cid_df.created_link_status_name_to_associated == False][['name_cid', 'associated_address_cid']].values,
    batch_size=50)
if not res_status:
    pprint(json.loads(json.loads(res[-1])['raw_log'])[0])

In [8]:
ens_data_with_cid_df[ens_data_with_cid_df.name=='1inch.eth']

,name,owner,resolver_address,associated_address,contenthash,content_cid,name_cid,owner_cid,associated_address_cid,created_link_status_name_to_content
263758,1inch.eth,0x39B7FBC184Ea3D16135224f1d34D9d13b07D4f59,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x1111111254fb6c44bAC0beD2854e76F90643097d,e301017012200cc0896f348d7b86d5724f03cd8f91f1ee...,QmPCRt8v4iLrE8mgtPvYrDKj28jyoZMWdnGzXgQCBk59EV,QmSmTHbXGjW76SMVbNZGrgCjm3X2ndXs5iL5gCxZjQj7MB,QmWyatGmPEccj4T3RoupyCH8iAzosSHyMmUXF6LazXvJJc,QmYrN7qDEWZKgdd3KsvDk3J5Xs3CzfsM1pegH9NviDLKRt,True
